## Fast ai Lesson 3 

import packages

In [1]:
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function

Using gpu device 0: Tesla K80 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
path = '../data/dogscats/'
model_path = path + 'models/'

Load model

In [3]:
#load model, pop the last layer, and add in a normal dense layer with output_ndim = 2
model = vgg_ft(2)
#load the weights of finetune3
model.load_weights(model_path + 'finetune3.h5')
batch_size = 64

/home/ubuntu/anaconda2/lib/python2.7/site-packages/keras/layers/core.py:621: UserWarning: `output_shape` argument not specified for layer lambda_1 and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 224, 224)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))


In [4]:
#recall the summary of layers 
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 3, 224, 224)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
zeropadding2d_1 (ZeroPadding2D)  (None, 3, 226, 226)   0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 64, 224, 224)  1792        zeropadding2d_1[0][0]            
____________________________________________________________________________________________________
zeropadding2d_2 (ZeroPadding2D)  (None, 64, 226, 226)  0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [5]:
layers = model.layers
conv_idx = [index for index,layer in enumerate(layers) if type(layer) is Convolution2D]
last_conv_idx = conv_idx[-1]

In [6]:
last_conv_idx

30

In [7]:
#extract out all the layers prior to that 
conv_layers = layers[:last_conv_idx+1]
conv_model = Sequential(conv_layers)
#remaining dense layers 
fc_layers = layers[last_conv_idx+1:]

In [8]:
batch_size = 64
batches = get_batches(path+'train', shuffle=False, batch_size=batch_size)
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

val_classes = val_batches.classes
trn_classes = batches.classes
val_labels = onehot(val_classes)
trn_labels = onehot(trn_classes)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


Extract the convolutional output. it should be 14x14x512, right before the maxpooling

In [ ]:
trn_features = conv_model.predict_generator(batches, batches.nb_sample)
val_features = conv_model.predict_generator(val_batches, val_batches.nb_sample)
save_array(model_path + 'train_convlayer_features.bc', trn_features)
save_array(model_path + 'valid_convlayer_features.bc', val_features)

In [9]:
trn_features = load_array(model_path+'train_convlayer_features.bc')
val_features = load_array(model_path+'valid_convlayer_features.bc')

In [10]:
trn_features.shape


(23000, 512, 14, 14)

In [11]:
# NB: Since we're removing dropout, we want to half the weights
def proc_wgts(layer): return [o/2 for o in layer.get_weights()]
opt = RMSprop(lr = 0.00001, rho = 0.7) #rho is momentum

In [12]:
def get_fc_model():
    
    model = Sequential([
        MaxPooling2D(input_shape = conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.),
        Dense(4096, activation='relu'),
        Dropout(0,),
        Dense(2, activation= 'softmax')
        
    ])
    
    #copy the fc_layers to the model above, after half-ing the weights.
    for l1,l2 in zip(model.layers, fc_layers): l1.set_weights(proc_wgts(l2))
    
    model.compile(optimizer= opt, loss='categorical_crossentropy',metrics = ['accuracy'])
    
    return model

In [13]:
fc_model = get_fc_model()

In [14]:
fc_model.fit(x = trn_features,
             y = trn_labels,
             nb_epoch = 8, 
             batch_size = batch_size, 
             validation_data = (val_features,val_labels))

Train on 23000 samples, validate on 2000 samples
Epoch 1/8
23000/23000 [==============================] - 23s - loss: 0.2698 - acc: 0.9808 - val_loss: 15.1390 - val_acc: 0.0575

In [15]:
fc_model.save_weights(model_path + 'no_dropout.h5')
fc_model.load_weights(model_path + 'no_dropout.h5')

## Data Augmentation

Remove width_zoom_range

In [16]:
gen = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.1, 
                               height_shift_range=0.1, zoom_range=0.1, horizontal_flip=True)

In [17]:
batches = get_batches(path+'train', gen, batch_size=batch_size)
# NB: We don't want to augment or shuffle the validation set
val_batches = get_batches(path+'valid', shuffle=False, batch_size=batch_size)

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [18]:
fc_model = get_fc_model()

for layer in conv_model.layers: layer.trainable = False
# Look how easy it is to connect two models together!
conv_model.add(fc_model)

In [19]:
conv_model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])


In [20]:
conv_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=8, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

Epoch 1/8
23000/23000 [==============================] - 629s - loss: 0.3536 - acc: 0.9753 - val_loss: 0.2956 - val_acc: 0.9795

In [21]:
conv_model.save_weights(model_path + 'aug1.h5')

In [22]:
conv_model.load_weights(model_path + 'aug1.h5')

In [23]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        Dense(4096, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        Dense(1000, activation='softmax')
        ]

In [ ]:
p=0.6
bn_model = Sequential(get_bn_layers(0.6))
bn_model.pop()
for layer in bn_model.layers: layer.trainable=False
bn_model.add(Dense(2,activation='softmax'))
bn_model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(trn_features, trn_labels, nb_epoch=8, validation_data=(val_features, val_labels),verbose=2)


In [ ]:
bn_layers = get_bn_layers(0.6)
bn_layers.pop()
bn_layers.append(Dense(2,activation='softmax'))             
                 
final_model = Sequential(conv_layers)
for layer in final_model.layers: layer.trainable = False
for layer in bn_layers: final_model.add(layer)

In [ ]:
for l1,l2 in zip(bn_model.layers, bn_layers):
    l2.set_weights(l1.get_weights())

In [ ]:
final_model.compile(optimizer=Adam(), 
                    loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
final_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=1, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)

final_model.save_weights(model_path + 'final1.h5')


In [ ]:
final_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=4, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
final_model.save_weights(model_path + 'final2.h5')


In [ ]:
final_model.optimizer.lr=0.001


In [ ]:
final_model.fit_generator(batches, samples_per_epoch=batches.nb_sample, nb_epoch=4, 
                        validation_data=val_batches, nb_val_samples=val_batches.nb_sample)
bn_model.save_weights(model_path + 'final3.h5')


## Other attempts 

Tried to use the fc-layer above and transfer the weights and adding the batchnormalization

Required to change the code such that dropout is after batchnorm

In [ ]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(p),
        BatchNormalization(),
        Dense(4096, activation='relu'),
        Dropout(p),        
        BatchNormalization(),
        Dense(2, activation='softmax')
        ]

In [ ]:
p = 0.6
bn_model = Sequential(get_bn_layers(p))   


In [ ]:
def proc_wgts(layer, prev_p, new_p):
    scal = (1-prev_p)/(1-new_p)
    return [o*scal for o in layer.get_weights()]


In [ ]:
fitted_fc_layers = fc_model.layers[:4]+\
[BatchNormalization()]+fc_model.layers[4:6]+[BatchNormalization()]+fc_model.layers[6:]

In [ ]:
for l1,l2 in zip(bn_model.layers, fitted_fc_layers):
    if type(l1)==Dense: l1.set_weights(proc_wgts(l2, 0, p))

Set verbose = 2 to prevent the juytper notebook from crashing 

Tried trainable = True as well

In [ ]:
bn_model.pop()
for layer in bn_model.layers: layer.trainable=False
bn_model.add(Dense(2,activation='softmax'))
bn_model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])
bn_model.fit(trn_features, trn_labels, nb_epoch=8, validation_data=(val_features, val_labels), verbose = 2)
bn_model.save_weights(model_path + 'bn_do3_1.h5')

## Other attempts 

Tried to use the vggbn.py in the github repo and transfer the weights.

Required to change the code such that dropout is after batchnorm

### download model
Get the script vgg_bn [here](https://github.com/fastai/courses/blob/master/deeplearning1/nbs/vgg16bn.py) and place it in the same working directory

Note that the old drop out is 0.5


In [ ]:
def get_bn_layers(p):
    return [
        MaxPooling2D(input_shape=conv_layers[-1].output_shape[1:]),
        Flatten(),
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),        
        Dense(4096, activation='relu'),
        BatchNormalization(),
        Dropout(p),        
        Dense(2, activation='softmax')
        ]

In [ ]:
from vgg16bn import Vgg16BN
vgg2 = Vgg16BN()
model2 = vgg2.model 


In [ ]:
#refit model2
model2.pop()
for layer in model.layers: layer.trainable = False
model.add(Dense(2,activation='softmax'))


In [ ]:
layers2 = model2.layers
fc_layers2 = layers2[last_conv_idx+1:]

In [ ]:
bn_model = Sequential(get_bn_layers(0.6))

def proc_wgts(layer, prev_p, new_p):
    scal = (1-prev_p)/(1-new_p)
    return [o*scal for o in layer.get_weights()]

for l1,l2 in zip(bn_model.layers, fc_layers2): l1.set_weights(proc_wgts(l2,0.5,0.6))
    
bn_model.pop()
bn_model.add(Dense(2,activation='softmax'))
bn_model.compile(Adam(), 'categorical_crossentropy', metrics=['accuracy'])


Set verbose = 2 as it keep crashing due to the exchange of print from server

In [ ]:
bn_model.fit(trn_features, trn_labels, nb_epoch=8, validation_data=(val_features, val_labels), verbose=2) 
